In [3]:
pip install pandas

  Using cached pandas-2.3.2-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.3.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
Using cached pandas-2.3.2-cp311-cp311-win_amd64.whl (11.3 MB)
Using cached numpy-2.3.3-cp311-cp311-win_amd64.whl (13.1 MB)
Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached six-1.17.0-py2.py3-none-any.whl (11 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)

   ---------------------------------------- 0/6 [pytz]
   ---------------------------------------- 0/6 [pytz]
   ---------------------------------------- 0/6 [pytz]
   ------ --------------------------------- 1/6 [tzdat

In [5]:
pip install dotenv



   ---------------------------------------- 2/2 [dotenv]

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import necessary libraries
import chromadb
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Get vectorstore path from environment
vectorstore_path = os.getenv("VECTORSTORE_PATH")
if not vectorstore_path:
    vectorstore_path = "../../chroma_db_eachfileisanode"  # Default path

print(f"Vectorstore path: {vectorstore_path}")
print(f"Path exists: {os.path.exists(vectorstore_path)}")


Vectorstore path: D:\CODE\dsc2025\dsc2025API\src\chroma_db_master_program
Path exists: True


In [2]:
# Connect to ChromaDB and check collections
try:
    # Initialize ChromaDB client
    db = chromadb.PersistentClient(path=vectorstore_path)
    
    # List all collections
    collections = db.list_collections()
    print(f"Total collections found: {len(collections)}")
    print("\nCollections:")
    for i, collection in enumerate(collections):
        print(f"{i+1}. {collection.name}")
    
except Exception as e:
    print(f"Error connecting to ChromaDB: {e}")
    print("Please check if the vectorstore path is correct and ChromaDB is properly initialized.")


Total collections found: 3

Collections:
1. question_collection
2. QA_collection
3. question_answer_collection


In [3]:
# Check question_answer_collection specifically
try:
    # Get or create the question_answer_collection
    question_collection = db.get_or_create_collection("question_answer_collection")
    
    # Count total documents in the collection
    total_count = question_collection.count()
    print(f"\n📊 QUESTION ANSWER COLLECTION STATISTICS:")
    print(f"Total questions in collection: {total_count}")
    
    if total_count > 0:
        # Get a sample of documents to see the structure
        sample_docs = question_collection.get(limit=3)
        
        print(f"\n📝 SAMPLE DOCUMENTS (first 3):")
        for i, (doc_id, doc_text, metadata) in enumerate(zip(sample_docs['ids'], sample_docs['documents'], sample_docs['metadatas'])):
            print(f"\n--- Document {i+1} ---")
            print(f"ID: {doc_id}")
            print(f"Text: {doc_text[:200]}..." if len(doc_text) > 200 else f"Text: {doc_text}")
            print(f"Metadata: {metadata}")
    
except Exception as e:
    print(f"Error accessing question_answer_collection: {e}")



📊 QUESTION ANSWER COLLECTION STATISTICS:
Total questions in collection: 137

📝 SAMPLE DOCUMENTS (first 3):

--- Document 1 ---
ID: AI_engineer-0
Text: AI khác biệt với lập trình truyền thống như thế nào?
Metadata: {'answer': 'Trí tuệ Nhân tạo (AI) là khi các cỗ máy, đặc biệt là máy tính, được thiết kế để suy nghĩ và hành động như con người. AI giúp máy móc học hỏi từ thông tin, giải quyết vấn đề và tự cải thiện. Những khác biệt chính: Lập trình truyền thống mã hóa rõ ràng các quy tắc, còn AI học các mẫu từ dữ liệu; AI có thể thích nghi và cải thiện theo thời gian; và AI xử lý các vấn đề phi cấu trúc tốt hơn.', '_node_type': 'TextNode', 'document_id': 'None', 'question': 'AI khác biệt với lập trình truyền thống như thế nào?', 'doc_id': 'None', 'index': 0, 'source': 'AI_engineer', 'ref_doc_id': 'None', '_node_content': '{"id_": "AI_engineer-0", "embedding": null, "metadata": {"source": "AI_engineer", "question": "AI kh\\u00e1c bi\\u1ec7t v\\u1edbi l\\u1eadp tr\\u00ecnh truy\\u1ec1n th\\

In [4]:
# Analyze metadata to understand question distribution
try:
    if total_count > 0:
        # Get all documents to analyze metadata
        all_docs = question_collection.get()
        
        # Count by source
        source_counts = {}
        index_counts = {}
        
        for metadata in all_docs['metadatas']:
            source = metadata.get('source', 'Unknown')
            index = metadata.get('index', 'Unknown')
            
            source_counts[source] = source_counts.get(source, 0) + 1
            index_counts[index] = index_counts.get(index, 0) + 1
        
        print(f"\n📈 METADATA ANALYSIS:")
        print(f"\nBy Source:")
        for source, count in sorted(source_counts.items()):
            print(f"  {source}: {count} questions")
        
        print(f"\nBy Index (first 10):")
        for index, count in sorted(index_counts.items())[:10]:
            print(f"  Index {index}: {count} questions")
        
        # Check if there are answers in metadata
        answers_found = 0
        for metadata in all_docs['metadatas']:
            if metadata.get('answer'):
                answers_found += 1
        
        print(f"\n💡 ANSWER ANALYSIS:")
        print(f"Documents with answers: {answers_found}/{total_count}")
        print(f"Answer coverage: {(answers_found/total_count)*100:.1f}%")
        
except Exception as e:
    print(f"Error analyzing metadata: {e}")



📈 METADATA ANALYSIS:

By Source:
  AI_engineer: 50 questions
  cloud_engineer: 87 questions

By Index (first 10):
  Index 0: 2 questions
  Index 1: 2 questions
  Index 2: 2 questions
  Index 3: 2 questions
  Index 4: 2 questions
  Index 5: 2 questions
  Index 6: 2 questions
  Index 7: 2 questions
  Index 8: 2 questions
  Index 9: 2 questions

💡 ANSWER ANALYSIS:
Documents with answers: 137/137
Answer coverage: 100.0%


In [5]:
# Test retrieval functionality
try:
    if total_count > 0:
        print(f"\n🔍 TESTING RETRIEVAL FUNCTIONALITY:")
        
        # Test queries
        test_queries = [
            "machine learning",
            "python programming", 
            "data structures",
            "algorithms",
            "deep learning"
        ]
        
        for query in test_queries:
            print(f"\nQuery: '{query}'")
            results = question_collection.query(
                query_texts=[query],
                n_results=3
            )
            
            if results['documents'][0]:
                print(f"  Found {len(results['documents'][0])} results:")
                for i, (doc, metadata) in enumerate(zip(results['documents'][0], results['metadatas'][0])):
                    print(f"    {i+1}. {doc[:100]}... (Source: {metadata.get('source', 'Unknown')})")
            else:
                print(f"  No results found")
                
except Exception as e:
    print(f"Error testing retrieval: {e}")



🔍 TESTING RETRIEVAL FUNCTIONALITY:

Query: 'machine learning'


C:\Users\ADMIN\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:16<00:00, 4.98MiB/s]


Error testing retrieval: Collection expecting embedding with dimension of 1536, got 384


In [6]:
# Summary report
print(f"\n📋 SUMMARY REPORT:")
print(f"=" * 50)
print(f"Vectorstore Path: {vectorstore_path}")
print(f"Path Exists: {os.path.exists(vectorstore_path)}")
print(f"Total Collections: {len(collections) if 'collections' in locals() else 'N/A'}")
print(f"Question Collection Count: {total_count if 'total_count' in locals() else 'N/A'}")
print(f"=" * 50)

if 'total_count' in locals() and total_count > 0:
    print(f"\n✅ Collection is ready for use!")
    print(f"✅ Total questions available: {total_count}")
    if 'answers_found' in locals():
        print(f"✅ Answer coverage: {(answers_found/total_count)*100:.1f}%")
else:
    print(f"\n❌ Collection is empty or not accessible")
    print(f"❌ Please check if questions have been properly embedded")



📋 SUMMARY REPORT:
Vectorstore Path: D:\CODE\dsc2025\dsc2025API\src\chroma_db_master_program
Path Exists: True
Total Collections: 3
Question Collection Count: 137

✅ Collection is ready for use!
✅ Total questions available: 137
✅ Answer coverage: 100.0%


In [7]:
# Check specifically for question_collection (not question_answer_collection)
try:
    print(f"\n🔍 CHECKING 'question_collection' (the one mentioned in your log):")
    
    # Try to get question_collection
    question_collection = db.get_or_create_collection("question_collection")
    
    # Count total documents in the collection
    total_count = question_collection.count()
    print(f"Total questions in 'question_collection': {total_count}")
    
    if total_count > 0:
        # Get a sample of documents to see the structure
        sample_docs = question_collection.get(limit=5)
        
        print(f"\n📝 SAMPLE DOCUMENTS from 'question_collection' (first 5):")
        for i, (doc_id, doc_text, metadata) in enumerate(zip(sample_docs['ids'], sample_docs['documents'], sample_docs['metadatas'])):
            print(f"\n--- Document {i+1} ---")
            print(f"ID: {doc_id}")
            print(f"Text: {doc_text[:200]}..." if len(doc_text) > 200 else f"Text: {doc_text}")
            print(f"Metadata: {metadata}")
    
except Exception as e:
    print(f"Error accessing 'question_collection': {e}")



🔍 CHECKING 'question_collection' (the one mentioned in your log):
Total questions in 'question_collection': 1768

📝 SAMPLE DOCUMENTS from 'question_collection' (first 5):

--- Document 1 ---
ID: AI_engineer-0
Text: AI khác biệt với lập trình truyền thống như thế nào?
Metadata: {'source': 'AI_engineer', 'question': 'AI khác biệt với lập trình truyền thống như thế nào?', 'ref_doc_id': 'None', 'document_id': 'None', 'doc_id': 'None', 'answer': 'Trí tuệ Nhân tạo (AI) là khi các cỗ máy, đặc biệt là máy tính, được thiết kế để suy nghĩ và hành động như con người. AI giúp máy móc học hỏi từ thông tin, giải quyết vấn đề và tự cải thiện. Những khác biệt chính: Lập trình truyền thống mã hóa rõ ràng các quy tắc, còn AI học các mẫu từ dữ liệu; AI có thể thích nghi và cải thiện theo thời gian; và AI xử lý các vấn đề phi cấu trúc tốt hơn.', '_node_content': '{"id_": "AI_engineer-0", "embedding": null, "metadata": {"source": "AI_engineer", "question": "AI kh\\u00e1c bi\\u1ec7t v\\u1edbi l\\u1eadp tr\

In [8]:
# Compare both collections
try:
    print(f"\n📊 COMPARING BOTH COLLECTIONS:")
    
    # Check question_answer_collection (used by chatbot_tools.py)
    qa_collection = db.get_or_create_collection("question_answer_collection")
    qa_count = qa_collection.count()
    
    # Check question_collection (mentioned in your log)
    q_collection = db.get_or_create_collection("question_collection")
    q_count = q_collection.count()
    
    print(f"'question_answer_collection': {qa_count} documents")
    print(f"'question_collection': {q_count} documents")
    
    # Check if they are the same collection with different names
    if qa_count == q_count and qa_count > 0:
        print(f"\n⚠️  Both collections have the same count. They might be the same data.")
        
        # Compare a few documents
        qa_sample = qa_collection.get(limit=3)
        q_sample = q_collection.get(limit=3)
        
        print(f"\nComparing first 3 documents:")
        for i in range(min(3, len(qa_sample['ids']))):
            if qa_sample['ids'][i] == q_sample['ids'][i]:
                print(f"  Document {i+1}: Same ID ✅")
            else:
                print(f"  Document {i+1}: Different IDs ❌")
                print(f"    qa_collection: {qa_sample['ids'][i]}")
                print(f"    q_collection: {q_sample['ids'][i]}")
    
except Exception as e:
    print(f"Error comparing collections: {e}")



📊 COMPARING BOTH COLLECTIONS:
'question_answer_collection': 137 documents
'question_collection': 1768 documents


In [ ]:
# Check the chatbot_tools.py configuration
print(f"\n🔧 CHECKING CHATBOT_TOOLS.PY CONFIGURATION:")
print(f"Vectorstore path in chatbot_tools.py: {vectorstore_path}")

# Check what collection name is used in chatbot_tools.py
try:
    # This simulates what chatbot_tools.py does
    db_test = chromadb.PersistentClient(path=vectorstore_path)
    chroma_collection = db_test.get_or_create_collection("question_answer_collection")
    
    print(f"Collection name used by chatbot_tools.py: 'question_answer_collection'")
    print(f"Count in this collection: {chroma_collection.count()}")
    
    # Check if there are any documents with 'automation_engineer' in metadata
    all_docs = chroma_collection.get()
    automation_docs = []
    
    for i, metadata in enumerate(all_docs['metadatas']):
        if metadata and 'source' in metadata:
            if 'automation' in metadata['source'].lower() or 'engineer' in metadata['source'].lower():
                automation_docs.append((i, metadata))
    
    print(f"\nDocuments with 'automation' or 'engineer' in source: {len(automation_docs)}")
    for i, (doc_idx, metadata) in enumerate(automation_docs[:5]):
        print(f"  {i+1}. Source: {metadata.get('source', 'Unknown')}")
        print(f"     Text: {all_docs['documents'][doc_idx][:100]}...")
    
except Exception as e:
    print(f"Error checking chatbot_tools configuration: {e}")


In [ ]:
# SOLUTION: Copy data from question_collection to question_answer_collection
print(f"\n🔧 SOLUTION: COPYING DATA TO CORRECT COLLECTION")

try:
    # Get source collection
    source_collection = db.get_or_create_collection("question_collection")
    source_count = source_collection.count()
    
    # Get target collection (the one used by chatbot_tools.py)
    target_collection = db.get_or_create_collection("question_answer_collection")
    target_count = target_collection.count()
    
    print(f"Source collection ('question_collection'): {source_count} documents")
    print(f"Target collection ('question_answer_collection'): {target_count} documents")
    
    if source_count > 0:
        # Get all documents from source
        all_source_docs = source_collection.get()
        
        print(f"\n📋 Copying {source_count} documents...")
        
        # Copy documents to target collection
        for i, (doc_id, doc_text, metadata) in enumerate(zip(all_source_docs['ids'], all_source_docs['documents'], all_source_docs['metadatas'])):
            try:
                # Add to target collection
                target_collection.add(
                    documents=[doc_text],
                    metadatas=[metadata],
                    ids=[doc_id]
                )
                if (i + 1) % 10 == 0:
                    print(f"  Copied {i + 1}/{source_count} documents...")
            except Exception as e:
                print(f"  Error copying document {doc_id}: {e}")
        
        # Verify the copy
        final_count = target_collection.count()
        print(f"\n✅ Copy completed!")
        print(f"Final count in 'question_answer_collection': {final_count}")
        
        # Test retrieval
        print(f"\n🔍 Testing retrieval...")
        test_results = target_collection.query(
            query_texts=["automation engineer"],
            n_results=3
        )
        
        if test_results['documents'][0]:
            print(f"Found {len(test_results['documents'][0])} results for 'automation engineer':")
            for i, doc in enumerate(test_results['documents'][0]):
                print(f"  {i+1}. {doc[:100]}...")
        else:
            print("No results found for 'automation engineer'")
            
    else:
        print("❌ Source collection is empty!")
        
except Exception as e:
    print(f"Error copying data: {e}")


In [ ]:
# ALTERNATIVE SOLUTION: Check what collection name should be used
print(f"\n🔧 ALTERNATIVE SOLUTION: CHECKING COLLECTION NAMES")

try:
    # List all collections to see what's available
    all_collections = db.list_collections()
    
    print(f"\n📋 All available collections:")
    for i, collection in enumerate(all_collections):
        count = collection.count()
        print(f"  {i+1}. '{collection.name}': {count} documents")
    
    # Check if we should modify chatbot_tools.py instead
    print(f"\n💡 RECOMMENDATION:")
    print(f"If 'question_collection' has the data you want to use:")
    print(f"1. Either copy data to 'question_answer_collection' (solution above)")
    print(f"2. Or modify chatbot_tools.py line 49:")
    print(f"   Change: chroma_collection = db.get_or_create_collection('question_answer_collection')")
    print(f"   To:     chroma_collection = db.get_or_create_collection('question_collection')")
    
except Exception as e:
    print(f"Error checking collections: {e}")


In [2]:
conda activate dsc2025


Note: you may need to restart the kernel to use updated packages.



CondaError: Run 'conda init' before 'conda activate'



In [1]:
conda init

no change     C:\Users\ADMIN\miniconda3\Scripts\conda.exe
no change     C:\Users\ADMIN\miniconda3\Scripts\conda-env.exe
no change     C:\Users\ADMIN\miniconda3\Scripts\conda-script.py
no change     C:\Users\ADMIN\miniconda3\Scripts\conda-env-script.py
no change     C:\Users\ADMIN\miniconda3\condabin\conda.bat
no change     C:\Users\ADMIN\miniconda3\Library\bin\conda.bat
no change     C:\Users\ADMIN\miniconda3\condabin\_conda_activate.bat
no change     C:\Users\ADMIN\miniconda3\condabin\rename_tmp.bat
no change     C:\Users\ADMIN\miniconda3\condabin\conda_auto_activate.bat
no change     C:\Users\ADMIN\miniconda3\condabin\conda_hook.bat
no change     C:\Users\ADMIN\miniconda3\Scripts\activate.bat
no change     C:\Users\ADMIN\miniconda3\condabin\activate.bat
no change     C:\Users\ADMIN\miniconda3\condabin\deactivate.bat
no change     C:\Users\ADMIN\miniconda3\Scripts\activate
no change     C:\Users\ADMIN\miniconda3\Scripts\deactivate
no change     C:\Users\ADMIN\miniconda3\etc\profile.d\

In [ ]:
pip install -r requirements.txt

In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import Settings, Document, VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
import chromadb
from typing import List, Dict, Any
from dotenv import load_dotenv
load_dotenv()


embeding_model_name = os.getenv("EMBEDDING_MODEL_NAME")
embeding_model_deployment_name = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
embedding_api_key = os.getenv("AZURE_OPENAI_EMBEDDING_API_KEY")
embedding_endpoint = os.getenv("AZURE_OPENAI_EMBEDDING_ENDPOINT")
embedding_api_version = os.getenv("AZURE_OPENAI_EMBEDDING_API_VERSION")

In [2]:
from llama_index.core import Settings

embed_model = AzureOpenAIEmbedding(
                model=embeding_model_name,
                deployment_name=embeding_model_deployment_name,
                api_key=embedding_api_key,
                azure_endpoint=embedding_endpoint,
                api_version=embedding_api_version,
            )

Settings.embed_model = embed_model

In [4]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("D:\CODE\dsc2025\dsc2025API\src\data").load_data()


<>:3: SyntaxWarning: invalid escape sequence '\C'
<>:3: SyntaxWarning: invalid escape sequence '\C'
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_15856\1106691481.py:3: SyntaxWarning: invalid escape sequence '\C'
  documents = SimpleDirectoryReader("D:\CODE\dsc2025\dsc2025API\src\data").load_data()


Failed to load file D:\CODE\dsc2025\dsc2025API\src\data\BusinessAnalyst.csv with error: No columns to parse from file. Skipping...
Failed to load file D:\CODE\dsc2025\dsc2025API\src\data\DataAnalyst.csv with error: No columns to parse from file. Skipping...
Failed to load file D:\CODE\dsc2025\dsc2025API\src\data\DataScience_QA.csv with error: 'utf-8' codec can't decode byte 0xe2 in position 5: invalid continuation byte. Skipping...
Failed to load file D:\CODE\dsc2025\dsc2025API\src\data\Game_QA.csv with error: Error tokenizing data. C error: Expected 3 fields in line 6, saw 4
. Skipping...
Failed to load file D:\CODE\dsc2025\dsc2025API\src\data\QA_Tester.csv with error: No columns to parse from file. Skipping...
Failed to load file D:\CODE\dsc2025\dsc2025API\src\data\Software_QA.csv with error: 'utf-8' codec can't decode byte 0x97 in position 20392: invalid start byte. Skipping...
Failed to load file D:\CODE\dsc2025\dsc2025API\src\data\SystemAdministrator.csv with error: No columns to 

Note: you may need to restart the kernel to use updated packages.


In [3]:
def create_documents_from_qa_data(df: pd.DataFrame, source_name: str) -> List[Document]:
    """Create Document objects from QA DataFrame"""
    documents = []
    
    for index, row in df.iterrows():
        # Create a combined text from question and answer
        if 'Question' in row and 'Answer' in row:
            # For DataScience QA format
            question = row['Question']
            answer = row['Answer']
            combined_text = f"Question: {question}\nAnswer: {answer}"
            
            # Create metadata
            metadata = {
                "source": source_name,
                "type": "qa_pair",
                "question": question,
                "answer": answer,
                "index": index
            }
        # Create Document object
        doc = Document(
            text=combined_text,
            metadata=metadata
        )
        documents.append(doc)
    
    print(f"Created {len(documents)} documents from {source_name}")
    return documents

In [4]:

def create_vector_store(collection_name: str, documents: List[Document], db_path: str = "./chroma_db_master_program"):
    """Create a vector store with the given documents"""
    try:
        # Initialize ChromaDB
        db = chromadb.PersistentClient(path=db_path)
        chroma_collection = db.get_or_create_collection(collection_name)
        
        # Create vector store
        vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
        storage_context = StorageContext.from_defaults(vector_store=vector_store)
        
        # Create index
        index = VectorStoreIndex(
            documents, 
            storage_context=storage_context
        )
        
        print(f"Successfully created vector store for collection: {collection_name}")
        print(f"Number of documents indexed: {len(documents)}")
        
        return index, vector_store
        
    except Exception as e:
        print(f"Error creating vector store for {collection_name}: {e}")
        return None, None

In [ ]:
import csv
with open('D:\cv_review_module-main\chatbot_sdh\chatbot_mi\src\data\DataScience_QA_vietnamese.csv', mode='r') as file:
    DS_QA = pd.read_csv(file)
    for row in DS_QA:
        print(row)

    if DS_QA is not None:
        ds_documents = create_documents_from_qa_data(DS_QA, "DataScience_QA")
        if ds_documents:
            ds_index, ds_vector_store = create_vector_store(
                "datascience_qa_collection", 
                ds_documents
            )

Question
Answer
Created 158 documents from DataScience_QA
Successfully created vector store for collection: datascience_qa_collection
Number of documents indexed: 158


In [ ]:
with open('D:\cv_review_module-main\chatbot_sdh\chatbot_mi\src\data\Software_QA_vietnamese.csv', mode='r') as file:
    DS_QA = pd.read_csv(file)
    for row in DS_QA:
        print(row)

    if DS_QA is not None:
        ds_documents = create_documents_from_qa_data(DS_QA, "Software_QA")
        if ds_documents:
            ds_index, ds_vector_store = create_vector_store(
                "software_qa_collection", 
                ds_documents
            )

Question Number
Question
Answer
Category
Difficulty


NameError: name 'create_documents_from_qa_data' is not defined

In [5]:
# Tạo Nodes thay vì Documents
from llama_index.core.schema import TextNode


def create_nodes_from_qa_data(df: pd.DataFrame, source_name: str) -> List[TextNode]:
    """Tạo danh sách TextNode từ DataFrame QA."""
    nodes: List[TextNode] = []
    if "Question" not in df.columns and "Câu hỏi" in df.columns:
        df = df.rename(columns={"Câu hỏi": "Question"})
    if "Answer" not in df.columns and "Trả lời" in df.columns:
        df = df.rename(columns={"Trả lời": "Answer"})


    for index, row in df.iterrows():
        if 'Question' not in row or 'Answer' not in row:
            continue

        question = row['Question']
        answer = row['Answer']

        text_parts = f"{question}"
        metadata: Dict[str, Any] = {
            "source": source_name,
            "question": question,
            "answer": answer,
            "index": int(index),
        }

        node = TextNode(text=text_parts, metadata=metadata, id_=f"{source_name}-{index}")
        nodes.append(node)

    print(f"Created {len(nodes)} nodes from {source_name}")
    return nodes


In [6]:
# Lưu Nodes vào Chroma thông qua VectorStoreIndex
from llama_index.core import StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from typing import List
# If you use TextNode, also ensure:
from llama_index.core.schema import TextNode

def store_nodes_in_chroma(nodes: List[TextNode], collection_name: str, db_path: str = "./chroma_db_master_program"):
    db = chromadb.PersistentClient(path=db_path)
    chroma_collection = db.get_or_create_collection(collection_name)
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    # Xây index từ nodes (không phải Documents)
    index = VectorStoreIndex(nodes, storage_context=storage_context)
    print(f"Stored {len(nodes)} nodes into collection: {collection_name}")
    return index


In [7]:
#chuyển file xlsx sang csv
import pandas as pd


In [6]:
import pandas as pd

ds_df = pd.read_csv('D:/CODE/dsc2025/dsc2025API/src/data/AIEngineer.csv')


In [7]:
ds_df.head(5)

,STT,Câu hỏi,Trả lời
0,1,AI khác biệt với lập trình truyền thống như th...,"Trí tuệ Nhân tạo (AI) là khi các cỗ máy, đặc b..."
1,2,Các nhánh chính của AI là gì?,Các nhánh chính của AI bao gồm: Machine Learni...
2,3,Sự khác biệt giữa AI mạnh và AI yếu là gì?,AI mạnh (AGI) là máy có khả năng áp dụng trí t...
3,4,Sự khác biệt giữa AI biểu tượng và AI kết nối ...,AI biểu tượng sử dụng quy tắc và logic rõ ràng...
4,5,Sự khác biệt giữa mô hình tham số và phi tham ...,Mô hình tham số có số tham số cố định (ví dụ: ...


In [8]:
ds_df['Question']

KeyError: 'Question'

In [12]:
# Fix encoding issues and update file paths
import pandas as pd
import chardet

def detect_and_read_csv(file_path):
    """Detect encoding and read CSV file"""
    try:
        # Try different encodings
        encodings = ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1', 'utf-16']
        for encoding in encodings:
            try:
                df = pd.read_csv(file_path, encoding=encoding)
                print(f"Successfully read {file_path} with {encoding}: {len(df)} rows")
                return df
            except Exception as e:
                continue
        
        # If all encodings fail, try with error handling
        df = pd.read_csv(file_path, encoding='utf-8', errors='ignore')
        print(f"Read {file_path} with error handling: {len(df)} rows")
        return df
        
    except Exception as e:
        print(f"Failed to read {file_path}: {e}")
        return None

# Test reading the problematic files
print("Testing CSV file reading...")
ds_df = detect_and_read_csv("D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\DataScience_QA_vietnamese.csv")
sw_df = detect_and_read_csv("D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\Software_QA_vietnamese.csv")


Testing CSV file reading...
Failed to read D:\CODE\dsc2025\dsc2025API\src\data\DataScience_QA_vietnamese.csv: read_csv() got an unexpected keyword argument 'errors'
Failed to read D:\CODE\dsc2025\dsc2025API\src\data\Software_QA_vietnamese.csv: read_csv() got an unexpected keyword argument 'errors'


In [22]:
# Code để chuyển đổi file Excel (.xlsx) sang CSV
import pandas as pd
import os
from pathlib import Path

def convert_xlsx_to_csv(xlsx_file_path, csv_file_path=None, sheet_name=0):
    """
    Chuyển đổi file Excel sang CSV
    
    Args:
        xlsx_file_path (str): Đường dẫn đến file Excel
        csv_file_path (str, optional): Đường dẫn file CSV output. Nếu None, sẽ tự tạo tên
        sheet_name (str/int): Tên sheet hoặc index sheet (mặc định là 0)
    
    Returns:
        str: Đường dẫn file CSV đã tạo
    """
    try:
        # Đọc file Excel
        df = pd.read_excel(xlsx_file_path, sheet_name=sheet_name)
        
        # Tạo tên file CSV nếu không được cung cấp
        if csv_file_path is None:
            xlsx_path = Path(xlsx_file_path)
            csv_file_path = xlsx_path.parent / f"{xlsx_path.stem}.csv"
        
        # Lưu thành CSV với encoding UTF-8
        df.to_csv(csv_file_path, index=False, encoding='utf-8')
        
        print(f"✅ Đã chuyển đổi: {xlsx_file_path} -> {csv_file_path}")
        print(f"   Số dòng: {len(df)}, Số cột: {len(df.columns)}")
        print(f"   Các cột: {list(df.columns)}")
        
        return str(csv_file_path)
        
    except Exception as e:
        print(f"❌ Lỗi khi chuyển đổi {xlsx_file_path}: {e}")
        return None

#convert_xlsx_to_csv(str("D:\CODE\dsc2025\dsc2025API\src\data\BusinessAnalyst.xlsx"), str("D:\CODE\dsc2025\dsc2025API\src\data\BusinessAnalyst.csv"))
def convert_all_xlsx_in_directory(directory_path, output_directory=None):
    """
    Chuyển đổi tất cả file Excel trong thư mục
    
    Args:
        directory_path (str): Đường dẫn thư mục chứa file Excel
        output_directory (str, optional): Thư mục output. Nếu None, sẽ lưu cùng thư mục
    
    Returns:
        list: Danh sách các file CSV đã tạo
    """
    directory = Path(directory_path)
    if output_directory:
        output_dir = Path(output_directory)
        output_dir.mkdir(exist_ok=True)
    else:
        output_dir = directory
    
    converted_files = []
    xlsx_files = list(directory.glob("*.xlsx"))
    
    print(f"🔍 Tìm thấy {len(xlsx_files)} file Excel trong {directory}")
    
    for xlsx_file in xlsx_files:
        csv_file = output_dir / f"{xlsx_file.stem}.csv"
        result = convert_xlsx_to_csv(str(xlsx_file), str(csv_file))
        if result:
            converted_files.append(result)
    
    print(f"\n✅ Hoàn thành! Đã chuyển đổi {len(converted_files)} file")
    return converted_files

# Ví dụ sử dụng
print("=== CHUYỂN ĐỔI FILE EXCEL SANG CSV ===")
print("1. Chuyển đổi tất cả file Excel trong thư mục data:")
converted_files = convert_all_xlsx_in_directory("D:\\CODE\\dsc2025\\dsc2025API\\src\\data")


=== CHUYỂN ĐỔI FILE EXCEL SANG CSV ===
1. Chuyển đổi tất cả file Excel trong thư mục data:
🔍 Tìm thấy 40 file Excel trong D:\CODE\dsc2025\dsc2025API\src\data
✅ Đã chuyển đổi: D:\CODE\dsc2025\dsc2025API\src\data\AIEngineer.xlsx -> D:\CODE\dsc2025\dsc2025API\src\data\AIEngineer.csv
   Số dòng: 50, Số cột: 3
   Các cột: ['STT', 'Câu hỏi', 'Trả lời']
✅ Đã chuyển đổi: D:\CODE\dsc2025\dsc2025API\src\data\Automation.xlsx -> D:\CODE\dsc2025\dsc2025API\src\data\Automation.csv
   Số dòng: 19, Số cột: 3
   Các cột: ['STT', 'Câu hỏi', 'Trả lời']
✅ Đã chuyển đổi: D:\CODE\dsc2025\dsc2025API\src\data\Backend_QA.xlsx -> D:\CODE\dsc2025\dsc2025API\src\data\Backend_QA.csv
   Số dòng: 86, Số cột: 3
   Các cột: ['STT', 'Câu hỏi', 'Trả lời']
✅ Đã chuyển đổi: D:\CODE\dsc2025\dsc2025API\src\data\blockchain.xlsx -> D:\CODE\dsc2025\dsc2025API\src\data\blockchain.csv
   Số dòng: 50, Số cột: 3
   Các cột: ['STT', 'Câu hỏi', 'Trả lời']
✅ Đã chuyển đổi: D:\CODE\dsc2025\dsc2025API\src\data\BusinessAnalyst.xlsx -> D

In [12]:
# Kiểm tra tất cả collections trong ChromaDB
print("\n=== KIỂM TRA TẤT CẢ COLLECTIONS ===")

def check_all_collections(db_path: str = "./chroma_db_master_program"):
    """Kiểm tra tất cả collections trong ChromaDB"""
    try:
        db = chromadb.PersistentClient(path=db_path)
        collections = db.list_collections()
        
        print(f"📊 Tổng số collections: {len(collections)}")
        print("=" * 50)
        
        for collection in collections:
            collection_name = collection.name
            collection_info = collection.get()
            
            print(f"📁 Collection: {collection_name}")
            print(f"   - Số documents: {len(collection_info['ids'])}")
            
            if collection_info['metadatas']:
                # Lấy metadata keys
                metadata_keys = list(collection_info['metadatas'][0].keys())
                print(f"   - Metadata keys: {metadata_keys}")
                
                # Hiển thị một vài câu hỏi mẫu
                print(f"   - Câu hỏi mẫu:")
                for i, metadata in enumerate(collection_info['metadatas'][:2]):
                    question = metadata.get('question', 'N/A')
                    if len(question) > 80:
                        question = question[:80] + "..."
                    print(f"     {i+1}. {question}")
            
            print("-" * 30)
        
        return collections
        
    except Exception as e:
        print(f"❌ Lỗi khi kiểm tra collections: {e}")
        return None

# Kiểm tra tất cả collections
all_collections = check_all_collections()

print("\n🎯 Tóm tắt:")
if all_collections:
    for collection in all_collections:
        collection_info = collection.get()
        print(f"   - {collection.name}: {len(collection_info['ids'])} documents")



=== KIỂM TRA TẤT CẢ COLLECTIONS ===
📊 Tổng số collections: 1
📁 Collection: question_collection
   - Số documents: 0
------------------------------

🎯 Tóm tắt:
   - question_collection: 0 documents


In [19]:
# Pipeline tạo và lưu nodes từ CSV
# DataScience QA
import pandas as pd
# with open('D:\cv_review_module-main\chatbot_sdh\chatbot_mi\src\data\DataScience_QA_vietnamese.csv', mode='r') as file:
#     ds_df = pd.read_csv(file)
#     ds_nodes = create_nodes_from_qa_data(ds_df, "DataScience_QA")
#     _ = store_nodes_in_chroma(ds_nodes, "datascience")

# # Software QA
with open(r'D:/CODE/dsc2025/dsc2025API/src/data/AIEngineer.csv', mode='r') as file:
    sw_df = pd.read_csv(file)
    sw_nodes = create_nodes_from_qa_data(sw_df, "AI_engineer")
    _ = store_nodes_in_chroma(sw_nodes, "question_collection")


Created 50 nodes from AI_engineer
Stored 50 nodes into collection: question_collection


In [10]:

# Function để thêm câu hỏi từ CSV vào QA_collection dưới dạng TextNode
from pathlib import Path
from typing import Optional

def add_questions_from_csv(
    csv_file_path: str,
    collection_name: str = "QA_collection",
    source_name: Optional[str] = None,
    db_path: str = "./chroma_db_master_program",
) -> bool:
    """
    Đọc CSV bất kỳ, chuẩn hóa cột hỏi/đáp, tạo TextNode và lưu vào collection.

    Args:
        csv_file_path: Đường dẫn file CSV nguồn
        collection_name: Tên collection trong Chroma (mặc định: QA_collection)
        source_name: Nhãn nguồn trong metadata (mặc định: theo tên file)
        db_path: Đường dẫn thư mục ChromaDB
    Returns:
        True nếu thêm thành công, False nếu lỗi
    """
    try:
        # Đọc CSV với các encoding phổ biến
        encodings = ["utf-8", "utf-8-sig", "cp1252", "latin-1", "iso-8859-1"]
        df = None
        for enc in encodings:
            try:
                df = pd.read_csv(csv_file_path, encoding=enc)
                break
            except Exception:
                continue
        if df is None:
            df = pd.read_csv(csv_file_path, encoding="utf-8", errors="ignore")

        # Chuẩn hóa cột về 'Question' và 'Answer'
        rename_map_candidates = [
            {"Câu hỏi": "Question", "Trả lời": "Answer"},
            {"Câu hỏi (theo nguồn)": "Question", "Trả lời": "Answer"},
            {"Câu hỏi": "Question", "Câu trả lời": "Answer"},
            {"Bảng Câu Hỏi và Trả Lời Machine Learning": "Question", "Unnamed: 2": "Answer"},
            {"Trả lời (giữ nguyên)": "Answer"},
        ]
        for rename_map in rename_map_candidates:
            intersects = {k: v for k, v in rename_map.items() if k in df.columns}
            if intersects:
                df = df.rename(columns=intersects)

        # Chỉ giữ 2 cột cần thiết
        if "Question" in df.columns and "Answer" in df.columns:
            df = df[["Question", "Answer"]]
        else:
            print("❌ CSV cần có cột 'Question' và 'Answer' (sau chuẩn hóa). Cột hiện có:", list(df.columns))
            return False

        # Lọc dòng trống
        df = df.dropna(subset=["Question", "Answer"]).reset_index(drop=True)
        if df.empty:
            print("⚠️ Không có bản ghi hợp lệ sau khi lọc.")
            return False

        # Xác định nguồn
        if not source_name:
            source_name = Path(csv_file_path).stem

        # Tạo nodes và lưu
        nodes = create_nodes_from_qa_data(df, source_name)
        _ = store_nodes_in_chroma(nodes, collection_name, db_path)
        print(f"✅ Đã thêm {len(nodes)} nodes từ '{source_name}' vào collection '{collection_name}'.")
        return True

    except Exception as e:
        print(f"❌ Lỗi khi thêm dữ liệu từ CSV: {e}")
        return False

# Ví dụ: nạp thêm dữ liệu vào QA_collection



In [20]:
add_questions_from_csv(r"D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\Automation.csv", "question_collection", "automation_engineer")

Created 19 nodes from automation_engineer
Stored 19 nodes into collection: question_collection
✅ Đã thêm 19 nodes từ 'automation_engineer' vào collection 'question_collection'.


True

In [21]:
add_questions_from_csv(r"D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\blockchain.csv", "question_collection")

Created 50 nodes from blockchain
Stored 50 nodes into collection: question_collection
✅ Đã thêm 50 nodes từ 'blockchain' vào collection 'question_collection'.


True

In [22]:
add_questions_from_csv(r"D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\CloudEngineer.csv", "question_collection")

Created 87 nodes from CloudEngineer
Stored 87 nodes into collection: question_collection
✅ Đã thêm 87 nodes từ 'CloudEngineer' vào collection 'question_collection'.


True

In [23]:
add_questions_from_csv(r"D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\Cybersecurity.csv", "question_collection")

Created 46 nodes from Cybersecurity
Stored 46 nodes into collection: question_collection
✅ Đã thêm 46 nodes từ 'Cybersecurity' vào collection 'question_collection'.


True

In [24]:
add_questions_from_csv(r"D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\DataEngineer.csv", "question_collection")

Created 100 nodes from DataEngineer
Stored 100 nodes into collection: question_collection
✅ Đã thêm 100 nodes từ 'DataEngineer' vào collection 'question_collection'.


True

In [25]:
add_questions_from_csv(r"D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\DBAdministrator.csv", "question_collection")

Created 150 nodes from DBAdministrator
Stored 150 nodes into collection: question_collection
✅ Đã thêm 150 nodes từ 'DBAdministrator' vào collection 'question_collection'.


True

🔍 Tìm thấy 42 file CSV trong folder data

📄 [ 1/42] Đang xử lý: AIEngineer.csv
Created 50 nodes from AIEngineer
Stored 50 nodes into collection: question_collection
✅ Đã thêm 50 nodes từ 'AIEngineer' vào collection 'question_collection'.
✅ Thành công: AIEngineer.csv

📄 [ 2/42] Đang xử lý: Automation.csv
Created 19 nodes from Automation
Stored 19 nodes into collection: question_collection
✅ Đã thêm 19 nodes từ 'Automation' vào collection 'question_collection'.
✅ Thành công: Automation.csv

📄 [ 3/42] Đang xử lý: Backend_QA.csv
Created 86 nodes from Backend_QA
Stored 86 nodes into collection: question_collection
✅ Đã thêm 86 nodes từ 'Backend_QA' vào collection 'question_collection'.
✅ Thành công: Backend_QA.csv

📄 [ 4/42] Đang xử lý: BusinessAnalyst.csv
Created 31 nodes from BusinessAnalyst
Stored 31 nodes into collection: question_collection
✅ Đã thêm 31 nodes từ 'BusinessAnalyst' vào collection 'question_collection'.
✅ Thành công: BusinessAnalyst.csv

📄 [ 5/42] Đang xử lý: CloudEnginee

🔍 KIỂM TRA KẾT QUẢ CUỐI CÙNG
📊 Tổng số câu hỏi trong collection: 2561

📈 Phân bố theo nguồn dữ liệu:
----------------------------------------
  AIEngineer               :   50 câu hỏi
  Automation               :   19 câu hỏi
  Backend_QA               :   86 câu hỏi
  BusinessAnalyst          :   31 câu hỏi
  CloudEngineer            :   87 câu hỏi
  Cloude_computing         :   39 câu hỏi
  Cybersecurity            :   46 câu hỏi
  DBAdministrator          :  150 câu hỏi
  DataAnalyst              :   50 câu hỏi
  DataBase                 :   60 câu hỏi
  DataEngineer             :  100 câu hỏi
  DataScience              :  154 câu hỏi
  DevOps_QA                :   61 câu hỏi
  Fontend_QA               :  100 câu hỏi
  FullStack_QA             :  199 câu hỏi
  Game_QA                  :   30 câu hỏi
  ITProjectManager         :   46 câu hỏi
  IoT                      :   34 câu hỏi
  JavaSripts               :   51 câu hỏi
  MobileApp_QA             :   20 câu hỏi
  NLP             

In [26]:
add_questions_from_csv(r"D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\UserResearcher.csv", "question_collection")

Created 29 nodes from UserResearcher
Stored 29 nodes into collection: question_collection
✅ Đã thêm 29 nodes từ 'UserResearcher' vào collection 'question_collection'.


True

In [27]:
add_questions_from_csv(r"D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\Fontend_QA.csv", "question_collection", "frontend")

Created 100 nodes from frontend
Stored 100 nodes into collection: question_collection
✅ Đã thêm 100 nodes từ 'frontend' vào collection 'question_collection'.


True

In [28]:
add_questions_from_csv(r"D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\FullStack_QA.csv", "question_collection", "fullstack")

Created 199 nodes from fullstack
Stored 199 nodes into collection: question_collection
✅ Đã thêm 199 nodes từ 'fullstack' vào collection 'question_collection'.


True

In [31]:
add_questions_from_csv(r"D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\IoT.csv", "question_collection", "IoT")

Created 34 nodes from IoT
Stored 34 nodes into collection: question_collection
✅ Đã thêm 34 nodes từ 'IoT' vào collection 'question_collection'.


True

In [34]:
add_questions_from_csv(r"D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\ITProjectManager.csv", "question_collection")

Created 29 nodes from ITProjectManager
Stored 29 nodes into collection: question_collection
✅ Đã thêm 29 nodes từ 'ITProjectManager' vào collection 'question_collection'.


True

In [37]:
add_questions_from_csv(r"D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\ML.csv", "question_collection", "machinglearning")

Created 62 nodes from machinglearning
Stored 62 nodes into collection: question_collection
✅ Đã thêm 62 nodes từ 'machinglearning' vào collection 'question_collection'.


True

In [38]:
add_questions_from_csv(r"D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\MobileApp_QA.csv", "question_collection", "MobileApp")

Created 20 nodes from MobileApp
Stored 20 nodes into collection: question_collection
✅ Đã thêm 20 nodes từ 'MobileApp' vào collection 'question_collection'.


True

In [39]:
add_questions_from_csv(r"D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\NetworkEngineer.csv", "question_collection")

Created 49 nodes from NetworkEngineer
Stored 49 nodes into collection: question_collection
✅ Đã thêm 49 nodes từ 'NetworkEngineer' vào collection 'question_collection'.


True

In [40]:
add_questions_from_csv(r"D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\UserResearcher.csv", "question_collection")

Created 29 nodes from UserResearcher
Stored 29 nodes into collection: question_collection
✅ Đã thêm 29 nodes từ 'UserResearcher' vào collection 'question_collection'.


True

In [41]:
add_questions_from_csv(r"D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\VR.csv", "question_collection", "VRAREngineer")

Created 30 nodes from VRAREngineer
Stored 30 nodes into collection: question_collection
✅ Đã thêm 30 nodes từ 'VRAREngineer' vào collection 'question_collection'.


True

In [15]:
add_questions_from_csv(r"D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\Backend_QA.csv", "question_collection", "backend")

Created 86 nodes from backend
Stored 86 nodes into collection: question_collection
✅ Đã thêm 86 nodes từ 'backend' vào collection 'question_collection'.


True

In [19]:
add_questions_from_csv(r"D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\BusinessAnalyst.csv", "question_collection")

Created 31 nodes from BusinessAnalyst
Stored 31 nodes into collection: question_collection
✅ Đã thêm 31 nodes từ 'BusinessAnalyst' vào collection 'question_collection'.


True

In [20]:
add_questions_from_csv(r"D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\DataAnalyst.csv", "question_collection")

Created 50 nodes from DataAnalyst
Stored 50 nodes into collection: question_collection
✅ Đã thêm 50 nodes từ 'DataAnalyst' vào collection 'question_collection'.


True

In [21]:
add_questions_from_csv(r"D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\DataScience.csv", "question_collection")

Created 154 nodes from DataScience
Stored 154 nodes into collection: question_collection
✅ Đã thêm 154 nodes từ 'DataScience' vào collection 'question_collection'.


True

In [22]:
add_questions_from_csv(r"D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\ITProjectManager.csv", "question_collection")

Created 46 nodes from ITProjectManager
Stored 46 nodes into collection: question_collection
✅ Đã thêm 46 nodes từ 'ITProjectManager' vào collection 'question_collection'.


True

In [23]:
add_questions_from_csv(r"D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\QA_tester.csv", "question_collection")

Created 50 nodes from QA_tester
Stored 50 nodes into collection: question_collection
✅ Đã thêm 50 nodes từ 'QA_tester' vào collection 'question_collection'.


True

In [24]:
add_questions_from_csv(r"D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\SoftSkill.csv", "question_collection")

Created 32 nodes from SoftSkill
Stored 32 nodes into collection: question_collection
✅ Đã thêm 32 nodes từ 'SoftSkill' vào collection 'question_collection'.


True

In [25]:
add_questions_from_csv(r"D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\Software.csv", "question_collection", "SoftwareEngineer")

Created 200 nodes from SoftwareEngineer
Stored 200 nodes into collection: question_collection
✅ Đã thêm 200 nodes từ 'SoftwareEngineer' vào collection 'question_collection'.


True

In [26]:
add_questions_from_csv(r"D:\\CODE\\dsc2025\\dsc2025API\\src\\data\\UIUXDesigner.csv", "question_collection")

Created 33 nodes from UIUXDesigner
Stored 33 nodes into collection: question_collection
✅ Đã thêm 33 nodes từ 'UIUXDesigner' vào collection 'question_collection'.


True

❌ Lỗi khi xóa collection 'datascience_qa_collection': Database error: error returned from database: (code: 1) no such table: tenants


False